In [1]:
import torch 


In [3]:
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your
[0.55, 0.87, 0.66], # journey
[0.57, 0.85, 0.64], # starts
[0.22, 0.58, 0.33], # with
[0.77, 0.25, 0.10], # one
[0.05, 0.80, 0.55]] # step
) 

In [6]:
""" 
The first step of imprlementing self-attention is to compute the inntermediate values, 
(w) , refered to as attention scores, we determine these scores by computing the 
dot product of the query with every otehr input token: 
"""
query = inputs[1] 
atten_scores_2 = torch.empty(inputs.shape[0]) 
for i , x_i in enumerate(inputs): # we enumerate over the 0-th dimention 
    atten_scores_2[i] = torch.dot(x_i, query) 

print(atten_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [8]:
""" 
after calcuating attention scores we normalize them 
""" 
atten_weights_2_temp = atten_scores_2 / atten_scores_2.sum()
print("Attention weights:",atten_weights_2_temp) 
print("Sum:",atten_weights_2_temp.sum()) 

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [9]:
""" 
In practice , its more common and adviable to use softmax function for normalization. 
This approach is better at managing extreme valeus and offers more favorable 
gradient properteis during training. The following is a basic implementation of the 
softmax function for normalizin gthe attention scores : 
""" 
def softmax_naive(x): 
    return torch.exp(x) / torch.exp(x).sum(dim=0) 

In [10]:
atten_weights_2_naive = softmax_naive(atten_scores_2) 
print("Attention weights:",atten_weights_2_naive) 
print("Sum:",atten_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [12]:
attn_weights_2 = torch.softmax(atten_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [13]:
""" 
Now that we have computed the normalized attention weight , we are ready for the 
final step , caclulating the context vector z(2) by multiplying the embedded 
input token ,x(i), with the corresponding attention weights and then summing the 
resulting vectors. thus context vector z(2) is the weighte sum of all input vectors, 
obtained by multipying each input vector by its corresponding attention weight: 
""" 

query = inputs[1] 
context_vec_2 = torch.zeros(query.shape) 

for i, x_i in enumerate(inputs): 
    context_vec_2 += attn_weights_2[i]  *x_i 


print(context_vec_2 )


tensor([0.4419, 0.6515, 0.5683])


In [15]:
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [16]:
# dont use loop use matmul like this 
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [17]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [18]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [19]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [21]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key= torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [22]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4306, 1.4551])


In [23]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [24]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [25]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [27]:
"""" 
Now, we want to go from the atteon scores to the attention weights, we compute the 
attenion weights by scaling the attention scores and using the softmax function. 
Hower , now we scale the attention scores by dividing them by the square root of the 
embedding dimention of the key 
""" 
d_k = keys.shape[-1] 
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)
""" 
The reason for normalization by the embedding dimenstion size is to improve the training 
performance by avoding small gradients. For insce  when scaling up the embedding dimention, 
which is typically grater than 1,000 for gpt-lie LLMs, large dot product can result 
in very small gradients during backpropatation due to the softmax function, applied  to them . 
as dot products incrase the softmax function behaves more like a step function 
""" 


tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


' \nThe reason for normalization by the embedding dimenstion size is to improve the training \nperformance by avoding small gradients. For insce  when scaling up the embedding dimention, \nwhich is typically grater than 1,000 for gpt-lie LLMs, large dot product can result \nin very small gradients during backpropatation due to the softmax function, applied  to them . \nas dot products incrase the softmax function behaves more like a step function \n'

In [28]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


In [29]:
values.shape, attn_weights_2.shape
# 6 @ 6 x2 -> 2 

(torch.Size([6, 2]), torch.Size([6]))

In [30]:
import torch.nn as nn 


In [41]:
class SelfAttention_v1(nn.Module): 
    def __init__(self,d_in,d_out): 
        super().__init__() 
        self.w_query = nn.Parameter(torch.rand(d_in,d_out)) 
        self.w_key = nn.Parameter(torch.rand(d_in,d_out)) 
        self.w_value = nn.Parameter(torch.rand(d_in,d_out))
    
    def forward(self,x): 
        # x (6,3)
        #d_in = 3, d_out = 2
        keys = x @ self.w_key #(6,3) @ (3,2 )-> 6,2 
        queries = x @ self.w_query # 6,2
        values  = x @ self.w_value # 6,2
        attn_scores = queries @ keys.T # omega (6,2) @ (2,6) > 6,6 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5 , dim=-1

        ) # softmax -> sum , (6,6)
        context_vec = attn_weights @ values #(6,6) @ (6,2) -> 6,2 
        return context_vec 
        

In [42]:
torch.manual_seed(123) 
sa_v1 = SelfAttention_v1(d_in,d_out) 

In [43]:
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [35]:
inputs.shape

torch.Size([6, 3])

In [47]:
class SelfAttention_v2(nn.Module): 
    def __init__(self,d_in,d_out,qkv_bias=False): 
        super().__init__() 
        """
        self.w_query = nn.Parameter(torch.rand(d_in,d_out)) 
        self.w_key = nn.Parameter(torch.rand(d_in,d_out)) 
        self.w_value = nn.Parameter(torch.rand(d_in,d_out))
        """ 
        self.w_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_value = nn.Linear(d_in,d_out,bias=qkv_bias)

    
    def forward(self,x): 
        # x (6,3)
        #d_in = 3, d_out = 2
        keys = self.w_key(x)
        queries = self.w_query(x) # 6,2
        values  = self.w_value(x) # 6,2
        attn_scores = queries @ keys.T # omega (6,2) @ (2,6) > 6,6 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5 , dim=-1

        ) # softmax -> sum , (6,6)
        context_vec = attn_weights @ values #(6,6) @ (6,2) -> 6,2 
        return context_vec 
        

In [48]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [49]:
# Hiding future words with causal attention

queries = sa_v2.w_query(inputs)  
keys = sa_v2.w_key(inputs)  
attn_scores = queries @ keys.T 
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**5,dim=-1)
print(attn_weights)

tensor([[0.1677, 0.1666, 0.1666, 0.1662, 0.1669, 0.1660],
        [0.1682, 0.1667, 0.1667, 0.1659, 0.1667, 0.1658],
        [0.1682, 0.1667, 0.1667, 0.1659, 0.1667, 0.1658],
        [0.1675, 0.1667, 0.1667, 0.1662, 0.1667, 0.1662],
        [0.1674, 0.1667, 0.1667, 0.1663, 0.1666, 0.1663],
        [0.1678, 0.1667, 0.1667, 0.1661, 0.1667, 0.1661]],
       grad_fn=<SoftmaxBackward0>)


In [51]:
context_lenght  = attn_scores.shape[0]   
mask_simple = torch.tril(torch.ones(context_lenght,context_lenght)) #(T,T) 
print(mask_simple) 

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [53]:
masked_simple = attn_weights * mask_simple 
print(masked_simple)

tensor([[0.1677, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1682, 0.1667, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1682, 0.1667, 0.1667, 0.0000, 0.0000, 0.0000],
        [0.1675, 0.1667, 0.1667, 0.1662, 0.0000, 0.0000],
        [0.1674, 0.1667, 0.1667, 0.1663, 0.1666, 0.0000],
        [0.1678, 0.1667, 0.1667, 0.1661, 0.1667, 0.1661]],
       grad_fn=<MulBackward0>)


In [54]:
# i renomalize again 
row_sums = masked_simple.sum(dim=-1,keepdim=True) 
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5023, 0.4977, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3353, 0.3323, 0.3323, 0.0000, 0.0000, 0.0000],
        [0.2511, 0.2498, 0.2499, 0.2492, 0.0000, 0.0000],
        [0.2008, 0.1999, 0.1999, 0.1995, 0.1999, 0.0000],
        [0.1678, 0.1667, 0.1667, 0.1661, 0.1667, 0.1661]],
       grad_fn=<DivBackward0>)


In [57]:
mask = torch.triu(torch.ones(context_lenght,context_lenght),diagonal=1) 
masked = attn_scores.masked_fill(mask.bool(),-torch.inf) 
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [58]:
# now all we need to do is apply the sofmax function to these masked result and we are done 

attn_weights = torch.softmax(masked/ keys.shape[-1]**-0.5,dim=-1) 
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6023, 0.3977, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4289, 0.2850, 0.2861, 0.0000, 0.0000, 0.0000],
        [0.3031, 0.2411, 0.2416, 0.2142, 0.0000, 0.0000],
        [0.2360, 0.1961, 0.1964, 0.1777, 0.1937, 0.0000],
        [0.2232, 0.1649, 0.1654, 0.1417, 0.1654, 0.1393]],
       grad_fn=<SoftmaxBackward0>)


In [59]:
""" 
we could now use the modifed attention weights to comptue the context vectors via 
""" 
context_vec = attn_weights  @ values
print(context_vec)

tensor([[0.1855, 0.8812],
        [0.2689, 0.9299],
        [0.3032, 0.9453],
        [0.2965, 0.8643],
        [0.2689, 0.7604],
        [0.2742, 0.7676]], grad_fn=<MmBackward0>)


In [60]:
torch.manual_seed(123) 
dropout  = torch.nn.Dropout(0.5) 
example = torch.ones(6,6) 
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [61]:
""" 
When you apply dropout , to comesete for the reuction in active elements, teh values of the 
remaining elements in the matrix are scaled up by a factor of 1/0.5 = 2. this 
scaling is crucial to maintain the overall baclance of the attention weights, ensuring that 
the avearage influece of the attention mechanism remains consistent during 
both the training inference phases. 
now lets apply droput to the attention weight matrix itself: 
""" 
torch.manual_seed(123) 
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8579, 0.5700, 0.5722, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4823, 0.4831, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3923, 0.0000, 0.3555, 0.0000, 0.0000],
        [0.0000, 0.3299, 0.3308, 0.2835, 0.3308, 0.0000]],
       grad_fn=<MulBackward0>)


In [63]:
""" 
Having gained an understanding of causual attention and dropout masking , we can now develop 
a concise python class. This class id designed to facilitate the effecient application of these 
two techniques 
""" 

batch = torch.stack((inputs,inputs),dim=0)  
print(batch.shape)# simulating a batch 

torch.Size([2, 6, 3])


In [86]:
import torch
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout=0.2, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        # store mask as boolean for easier masking
        self.register_buffer('mask', mask.bool())

    def forward(self, x):  # x: (B, T, D_in)
        B, T, D_in = x.shape
        Q = self.W_query(x)   # (B, T, D_out)
        K = self.W_key(x)     # (B, T, D_out)
        V = self.W_value(x)   # (B, T, D_out)

        attn_scores = Q @ K.transpose(-2, -1)  # (B, T, T)
        attn_scores = attn_scores / (self.d_out ** 0.5)
        attn_scores = attn_scores.masked_fill(self.mask[:T, :T], float('-inf'))
        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context = attn_weights @ V  # (B, T, D_out)
        return context



In [89]:
# Reproducible example using the shapes you provided
torch.manual_seed(123)
batch = torch.randn(2, 6, 3)  # (B=2, T=6, D_in=3)
d_in = 3
d_out = 2
context_length = batch.shape[1]

ca = CausalAttention(d_in, d_out, context_length, dropout=0.0)
context_vecs = ca(batch)

print("batch.shape:", batch.shape)
print("context_vecs.shape:", context_vecs.shape)   # expected (2,6,2)


batch.shape: torch.Size([2, 6, 3])
context_vecs.shape: torch.Size([2, 6, 2])


In [95]:
# multiHead attetion , the long way :( 

class MultiHeadAttentionWrapper(nn.Module): 
    def __init__(self,d_in,d_out,
                context_length, 
                dropout, 
                num_heads, 
                qkv_bias=False
                ): 
        super().__init__() 
        self.heads = nn.ModuleList( 
            [
                CausalAttention(
                    d_in,d_out,context_lenght,dropout,qkv_bias
                )
                for _ in range(num_heads)
            ]
        )

    def forward(self,x): 
        return torch.cat([head(x) for head in self.heads],dim=-1) 


    



In [96]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2

In [97]:
mha = MultiHeadAttentionWrapper(
d_in, d_out, context_length, 0.0, num_heads=2
)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-3.8698e-02, -7.4725e-02,  4.0897e-02,  1.8001e-02],
         [-6.1939e-03,  4.4548e-02,  9.0551e-03, -9.7055e-04],
         [ 2.2642e-02,  1.9516e-01,  5.8530e-04, -8.6823e-02],
         [ 3.2217e-01, -2.5480e-01, -1.1349e-01, -8.4414e-02],
         [-3.0068e-02, -2.2620e-01, -4.1286e-02,  2.3767e-01],
         [ 4.4803e-02, -2.2442e-01, -1.0092e-02,  7.5202e-02]],

        [[-1.5634e-01, -5.3191e-01,  7.2012e-02,  4.2994e-01],
         [ 4.7026e-01, -2.5717e-01, -4.6930e-01, -3.4054e-02],
         [ 7.0893e-02, -3.4483e-01, -1.2255e-01,  1.4510e-01],
         [ 4.9815e-01, -1.4483e-01, -4.4064e-01, -5.8198e-02],
         [ 2.7034e-01, -1.5265e-01, -3.2781e-01, -6.1827e-02],
         [ 5.9032e-01, -1.0644e-01, -5.3756e-01, -2.0486e-01]]],
       grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [106]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)

        # ✅ fix typo and correct buffer
        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, T, D_in = x.shape

        Q = self.W_query(x)  # (B, T, D_out)
        K = self.W_key(x)    # (B, T, D_out)
        V = self.W_value(x)  # (B, T, D_out)

        # ✅ reshape for multi-heads
        Q = Q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)  # (B, num_heads, T, head_dim)
        K = K.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        # Attention scores
        attn_scores = (Q @ K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (B, num_heads, T, T)

        # ✅ causal mask
        mask = self.mask[:T, :T].bool()
        attn_scores = attn_scores.masked_fill(mask[None, None, :, :], float('-inf'))

        # Softmax and dropout
        attn_weights = torch.softmax(attn_scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Apply attention
        context = attn_weights @ V  # (B, num_heads, T, head_dim)

        # Merge heads back
        context = context.transpose(1, 2).contiguous().view(B, T, self.d_out)
        context = self.out_proj(context)

        return context


In [107]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.1829, 0.6533],
         [0.2044, 0.6853],
         [0.2343, 0.7194],
         [0.0991, 0.7718],
         [0.1481, 0.6130],
         [0.1328, 0.6741]],

        [[0.0987, 0.5343],
         [0.0386, 0.8978],
         [0.1073, 0.6658],
         [0.0658, 0.9106],
         [0.1098, 0.8067],
         [0.0572, 0.9746]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
